**3. Neural Network**

Neural networks can be constructed using the ``torch.nn`` package. 

An ``nn.Module`` contains layers, and a method ``forward(input)`` that returns ``output``. 

In [0]:
# Define the network 
import torch 
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        # 1 input image channel, 6 output image channels, 5x5 square convolutional kernel
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))     # (2, 2) window로 Max Pooling
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)          # size가 square면 편하게 숫자 하나만 써도 가능 
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x 

    def num_flat_features(self, x):
        size = x.size()[1:]             # all dimensions except batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features 

net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [0]:
# The learnable parameters of a model are returned by ``net.parameters()``

params = list(net.parameters())
print(len(params))
print(params[0].size())     # conv1d's weight

10
torch.Size([6, 1, 5, 5])


In [0]:
# Let us try a random 32 x 32 input. 
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)
print(input.size())

tensor([[-0.0535, -0.0913,  0.1465, -0.0316, -0.1683,  0.0250, -0.2473,  0.1078,
          0.0998, -0.1622]], grad_fn=<AddmmBackward>)
torch.Size([1, 1, 32, 32])


In [0]:
# Zero the gradient buffers of all params and backprops with random grads: 
net.zero_grad()
out.backward(torch.randn(1, 10))

**NOTE**

``torch.nn`` only supports mini-batches. The entire ``torch.nn`` package only supports inputs that are a mini-batch of samples, and not a single sample. 

For example, ``nn.Conv2d`` will take in a 4D Tensor of ``nSamples x nChannels x Height x Width``.

If you have a single sample, just use ``input.unsqueeze(0)`` to add a fake batch dimension.  

**Loss Function**

A loss function takes the (output, target) pair of inputs, and computes a value that estimates how far away the output is from the target. 

In [0]:
output = net(input)
target = torch.randn(10)
target = target.view(1, -1)         # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)


tensor(0.6852, grad_fn=<MseLossBackward>)


When we call ``loss.backward()``, the whole graph is differentiated w.r.t. the loss, and all Tensors in the graph that has ``requires_grad=True`` will have their ``.grad`` Tensor accumulated with the gradient 

In [0]:
print(loss.grad_fn)     # MSELoss
print(loss.grad_fn.next_functions[0][0])        # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])       # ReLU

**Backprop**

To backpropagate the error all we have to do is to ``loss.backward()``. 

You need to clear the existing gradients though, else gradients will be accumulated to existing gradients. 

In [0]:
net.zero_grad()                 # zeroes the gradient buffers of all parameters 

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([-1.7151e-02, -8.7118e-03, -1.2276e-02,  3.8387e-05, -7.8521e-03,
        -4.9987e-02])


**Update the weights**

The simplest update rule used in practice is the Stochastic Gradient Descent (SGD)

``weight = weight - learning rate * 
gradient``


In [0]:
# By Python code 

learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [0]:
# To use various different update rules such as SGD, Nesterov-SGD, Adam, RMSProp, etc. 

import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr = 0.01)

# training loop 안에서 
optimizer.zero_grad()           # zero the gradient buffers 
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()